In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Load the ArXiv dataset
arxiv_dataset = load_dataset('CShorten/ML-ArXiv-Papers', trust_remote_code=True)

# Combine title and abstract
def combine_title_abstract(example):
    example['combined_text'] = example['title'] + " " + example['abstract']
    return example

# Apply the text combination function
arxiv_dataset = arxiv_dataset['train'].map(combine_title_abstract)

# Select only the first 50,000 samples
max_samples = 50000
arxiv_dataset = arxiv_dataset.select(range(max_samples))

# Verify the dataset size
print(f"Dataset size after limiting: {len(arxiv_dataset)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

ML-Arxiv-Papers.csv:   0%|          | 0.00/147M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/117592 [00:00<?, ? examples/s]

Map:   0%|          | 0/117592 [00:00<?, ? examples/s]

Dataset size after limiting: 50000


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load tokenizer and model
model_checkpoint = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

# Define CLS pooling function
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

# Function to compute embeddings
def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**encoded_input)
    return cls_pooling(model_output)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [5]:
def compute_embeddings(batch):
    # Tokenize the entire batch at once
    encoded_input = tokenizer(batch["combined_text"], padding=True, truncation=True, return_tensors="pt")

    # Generate embeddings for the batch
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Pooling for the batch
    embeddings = cls_pooling(model_output)  # Or mean_pooling

    # Convert to NumPy arrays
    return {"embeddings": embeddings.detach().cpu().numpy()}

# Add embeddings to the dataset with efficient batching
arxiv_dataset = arxiv_dataset.map(compute_embeddings, batched=True, batch_size=512)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 71.1 MB/s eta 0:00:00


In [9]:
import numpy as np
import faiss

# Convert embeddings to a NumPy array
embeddings_np = np.vstack(arxiv_dataset["embeddings"]).astype(np.float32)  # Convert to float32

# Normalize embeddings for cosine similarity
faiss.normalize_L2(embeddings_np)

# Create and populate a FAISS index
dimension = embeddings_np.shape[1]  # Size of each embedding vector
index = faiss.IndexFlatIP(dimension)  # Use Inner Product for cosine similarity
index.add(embeddings_np)

# Save the FAISS index
faiss.write_index(index, "faiss_index.faiss")

print("FAISS index successfully created and saved!")


FAISS index successfully created and saved!


In [10]:
def semantic_search(query, top_k=5):
    # Compute query embedding
    query_embedding = get_embeddings([query]).detach().cpu().numpy()
    faiss.normalize_L2(query_embedding)

    # Search FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve results
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        results.append({
            "Title": arxiv_dataset['title'][idx],
            "Abstract": arxiv_dataset['abstract'][idx],
            "Similarity Score": round(dist, 4)
        })
    return results


In [13]:
def cls_pooling(model_output):
    # Use the [CLS] token embedding for the sentence
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    # Tokenize the input texts
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )

    # Compute embeddings using the model
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Pool embeddings
    return cls_pooling(model_output)


In [14]:
# Example query
query = "advancements in machine learning"

# Perform semantic search
results = semantic_search(query, top_k=5)

# Display the results
for result in results:
    print(f"Title: {result['Title']}")
    print(f"Abstract: {result['Abstract']}")
    print(f"Similarity Score: {result['Similarity Score']}")
    print("-" * 80)


Title: Not quite unreasonable effectiveness of machine learning algorithms
Abstract:   State-of-the-art machine learning algorithms demonstrate close to absolute
performance in selected challenges. We provide arguments that the reason can be
in low variability of the samples and high effectiveness in learning typical
patterns. Due to this fact, standard performance metrics do not reveal model
capacity and new metrics are required for the better understanding of
state-of-the-art.

Similarity Score: 0.9236000180244446
--------------------------------------------------------------------------------
Title: Optimization Problems for Machine Learning: A Survey
Abstract:   This paper surveys the machine learning literature and presents in an
optimization framework several commonly used machine learning approaches.
Particularly, mathematical optimization models are presented for regression,
classification, clustering, deep learning, and adversarial learning, as well as
new emerging application

In [11]:
model.save_pretrained("/content/drive/My Drive/Research Papers/saved_model")
tokenizer.save_pretrained("/content/drive/My Drive/Research Papers/saved_model")


('/content/drive/My Drive/Research Papers/saved_model/tokenizer_config.json',
 '/content/drive/My Drive/Research Papers/saved_model/special_tokens_map.json',
 '/content/drive/My Drive/Research Papers/saved_model/vocab.txt',
 '/content/drive/My Drive/Research Papers/saved_model/added_tokens.json',
 '/content/drive/My Drive/Research Papers/saved_model/tokenizer.json')

In [12]:
faiss.write_index(index, "/content/drive/My Drive/Research Papers/faiss_index.faiss")


In [15]:
import json

# Define the path to save the dataset
dataset_save_path = "/content/drive/My Drive/Research Papers/dataset.json"

# Prepare the data for saving
dataset_to_save = [
    {"title": item["title"], "abstract": item["abstract"]}
    for item in arxiv_dataset
]

# Save the dataset to a JSON file
with open(dataset_save_path, "w") as f:
    json.dump(dataset_to_save, f, indent=4)

print(f"Dataset saved to {dataset_save_path}")


Dataset saved to /content/drive/My Drive/Research Papers/dataset.json
